In [2]:
import csv
from itertools import islice 
import jieba
import re
#导入训练集
f = open('YoutubeCommentsSpam_train.csv', 'r')
d = csv.reader(f)

def not_empty(s):
    return s and s.strip()

# 逐行读入
tmp=[line for line in islice(d, 1, None)]

datatrain=[ ' '.join(list(filter(not_empty, list(jieba.cut(re.sub(r"[^\u4e00-\u9fa5a-zA-Z0-9]", " ", line[0]).lower().replace("\r\n","").strip(),cut_all=False))))) for line in tmp]

labeltrain=[line[1] for line in tmp]
print(datatrain)
f.close()
#导入测试集
f = open('YoutubeCommentsSpam_test.csv', 'r')
d = csv.reader(f)
# 逐行读入
tmp=[line for line in islice(d, 1, None)]
datatest=[ ' '.join(list(filter(not_empty, list(jieba.cut(re.sub(r"[^\u4e00-\u9fa5a-zA-Z0-9]", " ", line[0]).lower().replace("\r\n","").strip(),cut_all=False))))) for line in tmp]

labeltest=[line[1] for line in tmp]
# print(datatest)
f.close()


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\admin\AppData\Local\Temp\jieba.cache
Loading model cost 0.953 seconds.
Prefix dict has been built succesfully.


['447935454150 lovely girl talk to me xxx', 'i always end up coming back to this song br', 'my sister just received over 6 500 new a rel nofollow class ot hashtag href https plus google com s 23active active a youtube views right now the only thing she used was pimpmyviews com', 'cool', 'hello i am from palastine', 'go check out my rapping video called four wheels please 赂 聫', 'almost 1 billion', 'aslamu lykum from pakistan', 'eminem is idol for very people in espa 脙 卤 a and mexico or latinoamerica', 'help me get 50 subs please', 'i love song', 'alright ladies if you like this song then check out john rage 脗 聽 he s a smoking hot rapper coming into the game 脗 聽 he s not better than eminem lyrically but he s hotter hear some of his songs on my channel', 'the perfect example of abuse from husbands and the thing is i m a feminist so i definitely agree with this song and well if i see this someone s going to die just sayin', 'the boyfriend was charlie from the tv show lost', 'a href https w

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
 
#该类会将文本中的词语转换为词频矩阵，矩阵元素a[i][j] 表示j词在i类文本下的词频
vectorizer = CountVectorizer()

#该类会统计每个词语的tf-idf权值
tf_idf_transformer = TfidfTransformer()

#将文本转为词频矩阵并计算tf-idf
tf_idf = tf_idf_transformer.fit_transform(vectorizer.fit_transform(datatrain))
print(tf_idf)
#将tf-idf矩阵抽取出来，元素a[i][j]表示j词在i类文本中的tf-idf权重
datatrain_weight = tf_idf.toarray()
 
#对测试集进行tf-idf权重计算
tf_idf = tf_idf_transformer.transform(vectorizer.transform(datatest))
datatest_weight = tf_idf.toarray()  # 测试集TF-IDF权重矩阵
 
print('输出x_train文本向量：')
print(datatrain_weight)
print('输出x_test文本向量：')
print(datatest_weight)


  (0, 4246)	0.43605260774703497
  (0, 3838)	0.1600189046336432
  (0, 3731)	0.43605260774703497
  (0, 2506)	0.20252438737131848
  (0, 2418)	0.4536248055528404
  (0, 1758)	0.34418054458319797
  (0, 227)	0.478391427338851
  (1, 3955)	0.2793776151748986
  (1, 3838)	0.18575422607387065
  (1, 3807)	0.15293730581291157
  (1, 3522)	0.21129332662286557
  (1, 1403)	0.4903594012388117
  (1, 1000)	0.422608264243261
  (1, 779)	0.29030776559987237
  (1, 638)	0.34844320924155364
  (1, 483)	0.4412113666102488
  (2, 4279)	0.09878061019735332
  (2, 4092)	0.14698725126606946
  (2, 4031)	0.12620210707010351
  (2, 3973)	0.21454555543169473
  (2, 3802)	0.18990927944277736
  (2, 3782)	0.08072966499487602
  (2, 3455)	0.2274649565506764
  (2, 3398)	0.14065123379968505
  (2, 3240)	0.17462856387472883
  :	:
  (1848, 3289)	0.536396834687341
  (1849, 3529)	0.6757485857979463
  (1849, 3398)	0.4313263449366784
  (1849, 2133)	0.2904161516797752
  (1849, 667)	0.5224747762324521
  (1850, 4124)	0.6104682880953957
  (185

In [4]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()   
mnb.fit(datatrain_weight,labeltrain)    # 利用训练数据对模型参数进行估计
labeltestpredict = mnb.predict(datatest_weight)     # 对参数进行预测

In [5]:
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings("ignore")
print('精度是：', mnb.score(datatest_weight,labeltest))
print(classification_report(labeltest, labeltestpredict,digits=6))

精度是： 0.9421487603305785
              precision    recall  f1-score   support

           0   0.964286  0.931034  0.947368        58
           1   0.923077  0.967742  0.944882        62
1so beutiful   0.000000  0.000000  0.000000         1

    accuracy                       0.942149       121
   macro avg   0.629121  0.632925  0.630750       121
weighted avg   0.935201  0.942149  0.938265       121



In [24]:
from sklearn import svm
from sklearn import datasets
from sklearn.model_selection import train_test_split as ts

#import our data
iris = datasets.load_iris()
X = iris.data
y = iris.target

#split the data to  7:3
X_train,X_test,y_train,y_test = ts(X,y,test_size=0.3)

# select different type of kernel function and compare the score

# kernel = 'rbf'
clf_rbf = svm.SVC(kernel='rbf')
clf_rbf.fit(datatrain_weight,labeltrain)
score_rbf = clf_rbf.score(datatest_weight,labeltest)
print("The score of rbf is : %f"%score_rbf)

# kernel = 'linear'
clf_linear = svm.SVC(kernel='linear')
clf_linear.fit(datatrain_weight,labeltrain)
score_linear = clf_linear.score(datatest_weight,labeltest)
print("The score of linear is : %f"%score_linear)

# kernel = 'poly'
clf_poly = svm.SVC(kernel='poly')
clf_poly.fit(datatrain_weight,labeltrain)
score_poly = clf_poly.score(datatest_weight,labeltest)
print("The score of poly is : %f"%score_poly)


The score of rbf is : 0.512397
The score of linear is : 0.942149
The score of poly is : 0.512397


In [47]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(max_depth=200) 
rfc = rfc.fit(datatrain_weight,labeltrain)
result = rfc.score(datatest_weight,labeltest)  
print(result)

0.9752066115702479


In [13]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
bdt = AdaBoostClassifier(DecisionTreeClassifier(max_depth=4, min_samples_split=50, min_samples_leaf=5),
                         algorithm="SAMME",
                         n_estimators=300, learning_rate=0.7)
bdt.fit(datatrain_weight, labeltrain)
labeltestpredict=bdt.predict(datatest_weight)
print(bdt.score(datatest_weight,labeltest))

0.9586776859504132


In [14]:
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings("ignore")
print(classification_report(labeltest, labeltestpredict,digits=6))

              precision    recall  f1-score   support

           0   0.935484  1.000000  0.966667        58
           1   0.983051  0.935484  0.958678        62
1so beutiful   0.000000  0.000000  0.000000         1

    accuracy                       0.958678       121
   macro avg   0.639512  0.645161  0.641781       121
weighted avg   0.952126  0.958678  0.954584       121

